The purpose of this notebook is to generate various ratings/rankings features for the teams in our dataset. We will generate elo ratings, pi ratings, rpi ratings, as well as league rankings, and use a neural network to create embeddings for the home and away teams which will serve as a ranking of the home strength and away strength of the home and away teams respectively. 

In order to speed up the calculations we will be creating these features for the subset of top European leagues. This will give us more than enough games to generate accurate ratings and give us a large training set to work with. We will be conducting an EDA on the generated features and will be comparing them using a single variable logistic regression model.

In [27]:
import pandas as pd
import numpy as np
import os
import glob
from ast import literal_eval
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from scipy.optimize import minimize
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, brier_score_loss, log_loss
from torch.optim.lr_scheduler import ExponentialLR
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
import optuna
import pickle
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier


In [12]:
df = pd.read_csv("df_opt.csv")
df = df[df['season'] != 2013]

features = ['home_win', 'away_win', 'pi_diff','gd_diff', 
            'elo_gls_diff', 'home_ip_scaled', 'away_ip_scaled',
       'home_ip', 'away_ip', 'ip_diff', 'season']

df = df[features].astype(float)

train = df[df['season'] != 2022.]
test = df[df['season'] == 2022.]

train = train.dropna()
test = test.dropna()

In [28]:
class Optimiser:
    
    def __init__(self, df, model_name, verbosity=False):
        self.df = df
        self.model_name = model_name
        self.verbosity = verbosity
        
        if self.model_name == "XGB":
            self.model = XGBClassifier()
        elif self.model_name == "LGBM":
            self.model = LGBMClassifier()
        elif self.model_name == "GBC":
            self.model = GradientBoostingClassifier()
        elif self.model_name == "CAT":
            self.model = CatBoostClassifier(verbose = verbosity)
            
    def process_df(self, df):
        
        df = df.astype(float)
        df = df[df['season'] != 2013]
        df = df.dropna()
        train = df[df['season'] != 2022.]
        test = df[df['season'] == 2022.]
        
        X_train = train.drop(['home_win', 'away_win', 'season'], axis=1)
        X_test = test.drop(['home_win', 'away_win', 'season'], axis=1)
        y_train = train['home_win']
        y_test = test['home_win']

        return X_train, X_test, y_train, y_test
    
    def objective(self, trial):


        if self.model_name == "XGB":
            params = {
                
            'booster': 'gbtree',
            'max_depth': trial.suggest_int('max_depth', 1, 10),
            'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
            'scale_pos_weight': (0.45/0.55),
            'subsample': trial.suggest_float('subsample', 0.2, 1.0, step=0.1),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0, step=0.1),
            'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10, log=True),
            'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10, log=True),
            'gamma': trial.suggest_float('gamma', 1e-5, 10, log=True),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
                
        }
            model = XGBClassifier(**params)

        elif self.model_name == "LGBM":
            
            params = {
                'boosting_type': 'gbdt',
                'max_depth': trial.suggest_int('max_depth', 1, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
                'subsample': trial.suggest_float('subsample', 0.2, 1.0, step=0.1),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0, step=0.1),
                'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10, log=True),
                'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10, log=True),
                'min_child_weight': trial.suggest_float('min_child_weight', 1e-5, 10, log=True)
            }

            model = LGBMClassifier(**params)
            
        elif self.model_name == "GBC":
            
            params = {
                'loss': 'deviance',
                'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
                'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
                'subsample': trial.suggest_float('subsample', 0.2, 1.0, step=0.1),
                'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
                'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 10),
                'max_depth': trial.suggest_int('max_depth', 1, 10),
                'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
                'random_state': 42
            }

            model = GradientBoostingClassifier(**params)
            
        elif self.model_name == "CAT":

            params = {
                'iterations': trial.suggest_int('iterations', 100, 1000, step=100),
                'depth': trial.suggest_int('depth', 1, 10),
                'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
                'random_seed': 42,
                'loss_function': 'Logloss',
                'eval_metric': 'Accuracy',
                'bootstrap_type': 'Bayesian',
                'subsample': trial.suggest_float('subsample', 0.2, 1.0, step=0.1),
                'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.2, 1.0, step=0.1),
                'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 10, log=True),
                'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),
                'verbose': self.verbosity
            }

            model = CatBoostClassifier(**params)
        
        X_train, X_test, y_train, y_test = self.process_df(self.df)
        model.fit(X_train, y_train)
        y_prob = model.predict_proba(X_test)
        auc = roc_auc_score(y_test, y_prob[:, 1])

        return auc

    def optimise(self):
        
        self.X_train, self.X_test, self.y_train, self.y_test = self.process_df(self.df)

        study = optuna.create_study(direction='maximize')
        study.optimize(self.objective, n_trials=100, show_progress_bar=self.verbosity)
        best_params = study.best_params
        
        if self.model_name == "XGB":
            best_model = XGBClassifier(**best_params)
            
        elif  self.model_name == "LGBM":
            best_model = LGBMClassifier(**best_params)
            
        kfold = StratifiedKFold(n_splits=5, shuffle=True)
        results = cross_val_score(self.model, self.X_train, self.y_train, cv=kfold, verbose=self.verbosity)
        
        print("Results of K-fold CV for", self.model, ":", results)
        print("Mean:", results.mean())
        print("Std:", results.std())

        best_model.fit(self.X_train, self.y_train, 
                       early_stopping_rounds=1000, 
                       eval_set=[(self.X_test, self.y_test)],
                       verbose=self.verbosity)

        y_pred_test = best_model.predict(self.X_test)
        y_prob_test = best_model.predict_proba(self.X_test)[:, 1]

        test_accuracy = accuracy_score(self.y_test, y_pred_test)
        test_f1 = f1_score(self.y_test, y_pred_test)

        test_roc_auc_score = roc_auc_score(self.y_test, y_prob_test)
        test_precision_score = precision_score(self.y_test, y_pred_test)

        print("Best Hyperparameters:", best_params)
        print("Test Accuracy:", test_accuracy)
        print("Test F1 Score:", test_f1)
        print("Test AUC:", test_roc_auc_score)
        print("Test Precision Score:", test_precision_score)

        return best_model


In [24]:
# lightgbm with odds data
optimiser = Optimiser(df, "LGBM", verbosity=False)
model = optimiser.optimise()
model_filename = "lgbm_home_win_with_odds.pkl"
with open(model_filename, "wb") as file:
    pickle.dump(model, file)

[I 2023-07-05 17:41:20,628] A new study created in memory with name: no-name-c3500b8b-184c-48f9-b29c-39f7749503e2
[I 2023-07-05 17:41:20,711] Trial 0 finished with value: 0.7377686523024926 and parameters: {'max_depth': 1, 'learning_rate': 0.05050775271354401, 'n_estimators': 200, 'subsample': 0.5, 'colsample_bytree': 0.9000000000000001, 'reg_lambda': 3.866367189790053e-05, 'reg_alpha': 0.96847707935293, 'min_child_weight': 0.02994246844842017}. Best is trial 0 with value: 0.7377686523024926.
[I 2023-07-05 17:41:20,855] Trial 1 finished with value: 0.7379349387410225 and parameters: {'max_depth': 1, 'learning_rate': 0.03509850493836054, 'n_estimators': 400, 'subsample': 0.4, 'colsample_bytree': 0.6000000000000001, 'reg_lambda': 0.00035244351487782335, 'reg_alpha': 0.037231809764158935, 'min_child_weight': 5.468859580600358e-05}. Best is trial 1 with value: 0.7379349387410225.
[I 2023-07-05 17:41:21,120] Trial 2 finished with value: 0.7540082805238699 and parameters: {'max_depth': 8, 'l

[I 2023-07-05 17:41:36,670] Trial 22 finished with value: 0.7493745669623997 and parameters: {'max_depth': 7, 'learning_rate': 0.018441525014402223, 'n_estimators': 100, 'subsample': 0.9000000000000001, 'colsample_bytree': 0.4, 'reg_lambda': 7.794644118140835, 'reg_alpha': 0.2900871827292046, 'min_child_weight': 1.4599401788169115}. Best is trial 17 with value: 0.7712513730460497.
[I 2023-07-05 17:41:37,509] Trial 23 finished with value: 0.7706213772708069 and parameters: {'max_depth': 10, 'learning_rate': 0.027003771608544776, 'n_estimators': 600, 'subsample': 1.0, 'colsample_bytree': 0.5, 'reg_lambda': 0.04403471800866459, 'reg_alpha': 1.7374797245443323, 'min_child_weight': 0.1774082686437504}. Best is trial 17 with value: 0.7712513730460497.
[I 2023-07-05 17:41:38,397] Trial 24 finished with value: 0.769453316434305 and parameters: {'max_depth': 10, 'learning_rate': 0.04381156880021076, 'n_estimators': 600, 'subsample': 0.8, 'colsample_bytree': 0.7, 'reg_lambda': 1.414890007055601,

[I 2023-07-05 17:41:57,277] Trial 44 finished with value: 0.7719337558090409 and parameters: {'max_depth': 4, 'learning_rate': 0.03498711140630237, 'n_estimators': 900, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.7, 'reg_lambda': 9.207977574869663, 'reg_alpha': 1.103292253381097, 'min_child_weight': 1.1933812634840795}. Best is trial 41 with value: 0.7720807773553021.
[I 2023-07-05 17:41:58,056] Trial 45 finished with value: 0.7707714406421632 and parameters: {'max_depth': 4, 'learning_rate': 0.05800106801961582, 'n_estimators': 1000, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.8, 'reg_lambda': 8.788406947618515, 'reg_alpha': 0.8834086200970356, 'min_child_weight': 1.106565369985108}. Best is trial 41 with value: 0.7720807773553021.
[I 2023-07-05 17:41:58,750] Trial 46 finished with value: 0.7718732572877058 and parameters: {'max_depth': 4, 'learning_rate': 0.038039043736443394, 'n_estimators': 900, 'subsample': 0.2, 'colsample_bytree': 0.9000000000000001, 'reg_

[I 2023-07-05 17:42:11,695] Trial 66 finished with value: 0.7572941275876638 and parameters: {'max_depth': 6, 'learning_rate': 0.08732771554300779, 'n_estimators': 900, 'subsample': 0.4, 'colsample_bytree': 0.8, 'reg_lambda': 2.2347037674842185, 'reg_alpha': 1.118817984485216, 'min_child_weight': 1.2476580812988047}. Best is trial 59 with value: 0.7727810730882974.
[I 2023-07-05 17:42:12,371] Trial 67 finished with value: 0.7717316434305027 and parameters: {'max_depth': 4, 'learning_rate': 0.04047840412612542, 'n_estimators': 800, 'subsample': 0.30000000000000004, 'colsample_bytree': 1.0, 'reg_lambda': 4.174249789021216, 'reg_alpha': 0.2679531577859429, 'min_child_weight': 0.38528739490160513}. Best is trial 59 with value: 0.7727810730882974.
[I 2023-07-05 17:42:13,347] Trial 68 finished with value: 0.7654367553865653 and parameters: {'max_depth': 5, 'learning_rate': 0.06616894060264919, 'n_estimators': 1000, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.9000000000000001, 're

[I 2023-07-05 17:42:26,419] Trial 88 finished with value: 0.7686516265314745 and parameters: {'max_depth': 5, 'learning_rate': 0.045126993397019796, 'n_estimators': 900, 'subsample': 0.4, 'colsample_bytree': 0.8, 'reg_lambda': 4.293249364643706, 'reg_alpha': 1.5765846205718312, 'min_child_weight': 4.334118076792393}. Best is trial 71 with value: 0.7733455006337135.
[I 2023-07-05 17:42:26,776] Trial 89 finished with value: 0.7384402196873681 and parameters: {'max_depth': 1, 'learning_rate': 0.027669168780014387, 'n_estimators': 900, 'subsample': 0.2, 'colsample_bytree': 0.9000000000000001, 'reg_lambda': 7.084302481183561, 'reg_alpha': 0.1345449538570042, 'min_child_weight': 2.5510624333084553}. Best is trial 71 with value: 0.7733455006337135.
[I 2023-07-05 17:42:27,920] Trial 90 finished with value: 0.7625017321504014 and parameters: {'max_depth': 6, 'learning_rate': 0.05471065086890164, 'n_estimators': 1000, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.8, 'reg_lambda': 2.501

Results of K-fold CV for LGBMClassifier() : [0.70732379 0.7001652  0.70787445 0.70594714 0.70201047]
Mean: 0.7046642076247487
Std: 0.003043299599803405


C:\Users\kiera\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\kiera\anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Best Hyperparameters: {'max_depth': 4, 'learning_rate': 0.03831628585040062, 'n_estimators': 900, 'subsample': 0.2, 'colsample_bytree': 0.8, 'reg_lambda': 1.6589792192011186, 'reg_alpha': 0.5750262101450965, 'min_child_weight': 5.16458108325335}
Test Accuracy: 0.7110655737704918
Test F1 Score: 0.6647646219686162
Test AUC: 0.7736158850866075
Test Precision Score: 0.7147239263803681


In [25]:
# xgb without odds data
optimiser = Optimiser(df, "XGB", verbosity=False)
model = optimiser.optimise()

model_filename = "xgb_home_win_with_odds.pkl"
with open(model_filename, "wb") as file:
    pickle.dump(model, file)


[I 2023-07-05 17:43:49,888] A new study created in memory with name: no-name-4162b388-d646-479e-b596-2d4e90ec9550
[I 2023-07-05 17:43:53,465] Trial 0 finished with value: 0.7556758766370933 and parameters: {'max_depth': 3, 'learning_rate': 0.013115152641434652, 'n_estimators': 1000, 'subsample': 0.6000000000000001, 'colsample_bytree': 1.0, 'reg_lambda': 0.17803358180236992, 'reg_alpha': 0.00013134638906976326, 'gamma': 0.0016536198180401554, 'min_child_weight': 5}. Best is trial 0 with value: 0.7556758766370933.
[I 2023-07-05 17:43:54,814] Trial 1 finished with value: 0.7474771440642163 and parameters: {'max_depth': 10, 'learning_rate': 0.02611725081303864, 'n_estimators': 200, 'subsample': 1.0, 'colsample_bytree': 0.2, 'reg_lambda': 0.017553666146364794, 'reg_alpha': 0.13551824116877184, 'gamma': 0.00013583791485704687, 'min_child_weight': 10}. Best is trial 0 with value: 0.7556758766370933.
[I 2023-07-05 17:43:57,173] Trial 2 finished with value: 0.7533167722855935 and parameters: {'

[I 2023-07-05 17:45:13,494] Trial 21 finished with value: 0.763410223912125 and parameters: {'max_depth': 5, 'learning_rate': 0.02076391283854109, 'n_estimators': 1000, 'subsample': 0.8, 'colsample_bytree': 0.4, 'reg_lambda': 0.5431015404376119, 'reg_alpha': 0.013196856199393281, 'gamma': 9.274531534298323, 'min_child_weight': 1}. Best is trial 16 with value: 0.7696256865230249.
[I 2023-07-05 17:45:20,607] Trial 22 finished with value: 0.7687678918462189 and parameters: {'max_depth': 7, 'learning_rate': 0.028972586639300506, 'n_estimators': 1000, 'subsample': 0.9000000000000001, 'colsample_bytree': 0.5, 'reg_lambda': 0.026836060891380305, 'reg_alpha': 0.014817261148075436, 'gamma': 3.6594270517713685, 'min_child_weight': 2}. Best is trial 16 with value: 0.7696256865230249.
[I 2023-07-05 17:45:27,159] Trial 23 finished with value: 0.7692268694550064 and parameters: {'max_depth': 7, 'learning_rate': 0.024694389511390292, 'n_estimators': 900, 'subsample': 0.9000000000000001, 'colsample_by

[I 2023-07-05 17:46:40,170] Trial 42 finished with value: 0.770552429235319 and parameters: {'max_depth': 3, 'learning_rate': 0.0985005781834045, 'n_estimators': 900, 'subsample': 1.0, 'colsample_bytree': 0.30000000000000004, 'reg_lambda': 0.16546677461501663, 'reg_alpha': 0.0006831624619559347, 'gamma': 0.7580954487919044, 'min_child_weight': 4}. Best is trial 31 with value: 0.7718354034643009.
[I 2023-07-05 17:46:42,632] Trial 43 finished with value: 0.7573133924799325 and parameters: {'max_depth': 2, 'learning_rate': 0.03560304073641522, 'n_estimators': 800, 'subsample': 0.9000000000000001, 'colsample_bytree': 0.4, 'reg_lambda': 1.1503736513745904, 'reg_alpha': 0.00041493469211161346, 'gamma': 1.598864281132709, 'min_child_weight': 1}. Best is trial 31 with value: 0.7718354034643009.
[I 2023-07-05 17:46:47,246] Trial 44 finished with value: 0.7607763413603718 and parameters: {'max_depth': 5, 'learning_rate': 0.047948310931628976, 'n_estimators': 1000, 'subsample': 0.6000000000000001

[I 2023-07-05 17:47:47,645] Trial 63 finished with value: 0.7690196873679764 and parameters: {'max_depth': 4, 'learning_rate': 0.07935541775705542, 'n_estimators': 1000, 'subsample': 1.0, 'colsample_bytree': 0.30000000000000004, 'reg_lambda': 0.8051877485804004, 'reg_alpha': 0.00525626413065774, 'gamma': 0.46291239890188707, 'min_child_weight': 7}. Best is trial 31 with value: 0.7718354034643009.
[I 2023-07-05 17:47:51,955] Trial 64 finished with value: 0.7691180397127165 and parameters: {'max_depth': 5, 'learning_rate': 0.06114779309909109, 'n_estimators': 900, 'subsample': 1.0, 'colsample_bytree': 0.4, 'reg_lambda': 0.4315395995409495, 'reg_alpha': 0.00048001514498976735, 'gamma': 1.1137364541813608, 'min_child_weight': 6}. Best is trial 31 with value: 0.7718354034643009.
[I 2023-07-05 17:47:55,653] Trial 65 finished with value: 0.769061934938741 and parameters: {'max_depth': 3, 'learning_rate': 0.08571035491527962, 'n_estimators': 1000, 'subsample': 0.9000000000000001, 'colsample_by

[I 2023-07-05 17:48:42,621] Trial 84 finished with value: 0.7597897760878749 and parameters: {'max_depth': 3, 'learning_rate': 0.04670322545655829, 'n_estimators': 500, 'subsample': 1.0, 'colsample_bytree': 0.4, 'reg_lambda': 0.019474053761100146, 'reg_alpha': 0.0005842725247262394, 'gamma': 3.6095367720218383, 'min_child_weight': 9}. Best is trial 72 with value: 0.7731109421208281.
[I 2023-07-05 17:48:45,094] Trial 85 finished with value: 0.7712696239966204 and parameters: {'max_depth': 4, 'learning_rate': 0.07429245538940114, 'n_estimators': 600, 'subsample': 1.0, 'colsample_bytree': 0.5, 'reg_lambda': 0.03544755753933173, 'reg_alpha': 0.0019216149884007034, 'gamma': 1.242631650492688, 'min_child_weight': 9}. Best is trial 72 with value: 0.7731109421208281.
[I 2023-07-05 17:48:47,517] Trial 86 finished with value: 0.7628366708914238 and parameters: {'max_depth': 4, 'learning_rate': 0.05392456448182267, 'n_estimators': 600, 'subsample': 1.0, 'colsample_bytree': 0.5, 'reg_lambda': 0.04

Results of K-fold CV for XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, predictor=None, random_state=None,
              reg_alpha=None, reg_lambda=None, ...) : [0.69961454 0.6847467  0.69575991 0.68309471 0.70035803]
Mean: 0.6927147774244544
Std: 0.007366610631281702


C:\Users\kiera\anaconda3\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Best Hyperparameters: {'max_depth': 3, 'learning_rate': 0.07452314631965326, 'n_estimators': 900, 'subsample': 1.0, 'colsample_bytree': 0.30000000000000004, 'reg_lambda': 0.09919734255846553, 'reg_alpha': 0.0002524212417811137, 'gamma': 1.371890617189692, 'min_child_weight': 9}
Test Accuracy: 0.7040983606557377
Test F1 Score: 0.6558627264061009
Test AUC: 0.7709735530207014
Test Precision Score: 0.7070914696813977


In [ ]:
# xgb without odds data
optimiser = Optimiser(df, "GBC", verbosity=False)
model = optimiser.optimise()

model_filename = "GBC_home_win_with_odds.pkl"
with open(model_filename, "wb") as file:
    pickle.dump(model, file)
    
    
# xgb without odds data
optimiser = Optimiser(df, "CAT", verbosity=False)
model = optimiser.optimise()

model_filename = "CAT_home_win_with_odds.pkl"
with open(model_filename, "wb") as file:
    pickle.dump(model, file)

[I 2023-07-05 17:51:33,725] A new study created in memory with name: no-name-957ff15d-5578-469a-aab2-9b887144e92a
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 17:51:40,179] Trial 0 finished with value: 0.7368098014364175 and parameters: {'learning_rate': 0.059775910369254254, 'n_estimators': 900, 'subsample': 0.2, 'min_samples_split': 4, 'min_samples_leaf': 9, 'max_depth': 5, 'max_features': 'sqrt'}. Best is trial 0 with value: 0.7368098014364175.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 17:51:41,639] Trial 1 finished with value: 0.725986

[I 2023-07-05 17:54:05,446] Trial 14 finished with value: 0.7504797634136037 and parameters: {'learning_rate': 0.027986637683448716, 'n_estimators': 800, 'subsample': 0.30000000000000004, 'min_samples_split': 8, 'min_samples_leaf': 3, 'max_depth': 3, 'max_features': 'log2'}. Best is trial 6 with value: 0.7506406421630757.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 17:54:16,263] Trial 15 finished with value: 0.7504621884241656 and parameters: {'learning_rate': 0.03706742513603658, 'n_estimators': 1000, 'subsample': 0.7, 'min_samples_split': 5, 'min_samples_leaf': 3, 'max_depth': 2, 'max_features': 'log2'}. Best is trial 6 with value: 0.7506406421630757.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 

C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 17:56:02,461] Trial 29 finished with value: 0.7547950992817912 and parameters: {'learning_rate': 0.04493502429970954, 'n_estimators': 500, 'subsample': 0.8, 'min_samples_split': 4, 'min_samples_leaf': 8, 'max_depth': 6, 'max_features': 'log2'}. Best is trial 28 with value: 0.7563173637515843.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 17:56:14,916] Trial 30 finished with value: 0.7594984368398817 and parameters: {'learning_rate': 0.0490372961854511, 'n_estimators': 400, 'subsample': 0.8, 'min_sam

[I 2023-07-05 17:58:24,014] Trial 43 finished with value: 0.7582539923954372 and parameters: {'learning_rate': 0.0338498685265353, 'n_estimators': 400, 'subsample': 0.9000000000000001, 'min_samples_split': 4, 'min_samples_leaf': 9, 'max_depth': 6, 'max_features': 'log2'}. Best is trial 30 with value: 0.7594984368398817.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 17:58:42,606] Trial 44 finished with value: 0.7586764681030841 and parameters: {'learning_rate': 0.040180890690744465, 'n_estimators': 500, 'subsample': 1.0, 'min_samples_split': 4, 'min_samples_leaf': 8, 'max_depth': 6, 'max_features': 'log2'}. Best is trial 30 with value: 0.7594984368398817.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name '

C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 18:02:27,230] Trial 58 finished with value: 0.7582526404731729 and parameters: {'learning_rate': 0.05915998204675285, 'n_estimators': 600, 'subsample': 1.0, 'min_samples_split': 8, 'min_samples_leaf': 2, 'max_depth': 4, 'max_features': 'log2'}. Best is trial 56 with value: 0.7628606675116181.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 18:02:42,387] Trial 59 finished with value: 0.7628390367553866 and parameters: {'learning_rate': 0.07684630565242746, 'n_estimators': 600, 'subsample': 1.0, 'min_sa

C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 18:05:35,484] Trial 73 finished with value: 0.7623002957329953 and parameters: {'learning_rate': 0.09748986636623776, 'n_estimators': 600, 'subsample': 1.0, 'min_samples_split': 9, 'min_samples_leaf': 1, 'max_depth': 4, 'max_features': 'log2'}. Best is trial 56 with value: 0.7628606675116181.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(
[I 2023-07-05 18:05:50,604] Trial 74 finished with value: 0.7585960287283482 and parameters: {'learning_rate': 0.09841364378087833, 'n_estimators': 600, 'subsample': 1.0, 'min_sa

[I 2023-07-05 18:09:16,286] Trial 87 finished with value: 0.7523288550908322 and parameters: {'learning_rate': 0.09999996459852539, 'n_estimators': 800, 'subsample': 1.0, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_depth': 3, 'max_features': 'sqrt'}. Best is trial 56 with value: 0.7628606675116181.
C:\Users\kiera\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:280: FutureWarning: The loss parameter name 'deviance' was deprecated in v1.1 and will be removed in version 1.3. Use the new parameter name 'log_loss' which is equivalent.
  warnings.warn(


In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

thresholds = [0.25, 0.3, 0.35, 0.40, 0.45, 0.5, 0.55, 0.60, 0.65, .70, .75, 0.8, 0.85, 0.9]

for t in thresholds:
    preds = []
    for i in range(len(y_prob_test)):
        if y_prob_test[i] >= t:
            preds.append(1.)
        else:
            preds.append(0.)
    
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)
    
    print(f"Threshold: {t}")
    print("Confusion Matrix:")
    print(cm)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("-----------------------")
    

Threshold: 0.25
Confusion Matrix:
[[ 466  846]
 [ 110 1014]]
Precision: 0.5451612903225806
Recall: 0.902135231316726
F1 Score: 0.6796246648793566
-----------------------
Threshold: 0.3
Confusion Matrix:
[[615 697]
 [164 960]]
Precision: 0.5793602896801449
Recall: 0.8540925266903915
F1 Score: 0.6903991370010787
-----------------------
Threshold: 0.35
Confusion Matrix:
[[745 567]
 [224 900]]
Precision: 0.6134969325153374
Recall: 0.800711743772242
F1 Score: 0.6947124662292551
-----------------------
Threshold: 0.4
Confusion Matrix:
[[854 458]
 [303 821]]
Precision: 0.6419077404222049
Recall: 0.7304270462633452
F1 Score: 0.6833125260091554
-----------------------
Threshold: 0.45
Confusion Matrix:
[[954 358]
 [373 751]]
Precision: 0.6771866546438232
Recall: 0.6681494661921709
F1 Score: 0.6726377071204657
-----------------------
Threshold: 0.5
Confusion Matrix:
[[1025  287]
 [ 437  687]]
Precision: 0.7053388090349076
Recall: 0.6112099644128114
F1 Score: 0.6549094375595805
-------------------

In [53]:
def get_profits(odds, preds, threshold, validation_set):

    profit = 0
    bets = 0
    odds = odds.reset_index(drop = True)

    for i in range(len(odds)):

        if preds[i] > threshold:
            bets += 1
            o = odds[i]
            bet = 1 * o 
            if validation_set[i] == 1:
                profit += bet - 1
            else:
                profit += -1
    return profit, bets, profit/bets*100

def get_value_strat1(odds, preds, threshold, validation_set):
    
    c1 = 0
    c2 = 0
    p1 = 0
    odds = odds.reset_index(drop = True)
    
    for i in range(len(preds)):
        if preds[i] > 1/odds[i]:
            if abs(preds[i] - 1/odds[i]) > threshold:
                if validation_set[i] == 1:
                    o = odds[i]
                    p1 += (1 * o)
                    c1 += 1
                else:
                    c2 += 1
                    p1 += -1
                    
    return p1, c1, c2, (p1/(c1+c2))*100

def get_value_strat2(odds, preds, threshold, validation_set):
    
    c1 = 0
    c2 = 0
    p1 = 0
    odds = odds.reset_index(drop = True)
    
    for i in range(len(preds)):
        if preds[i] < 1/odds[i]:
            if abs(preds[i] - 1/odds[i]) > threshold:
                if validation_set[i] == 0:
                    p = 1 - (1/odds[i])
                    p1 += (1 * (1/p)) - 1
                    c1 += 1
                else:
                    c2 += 1
                    p1 += -1
                    
    return p1, c1, c2, (p1/(c1+c2))*100

In [54]:
validation_set = y_test.reset_index(drop = True)

for t in [0.25, 0.3, 0.35, 0.40, 0.45, 0.5, 0.55, 0.60, 0.65, .70, .75, 0.8, 0.85, 0.9]:
    results = get_profits(odds, y_prob_test, t, validation_set)
    print(f"At threshold {t} for predictions: \n Profit: {results[0]} \n Number of bets: {results[1]} \n Return %: {results[2]} \n")

print(get_value_strat1(odds, preds, 0.05, validation_set))
print(get_value_strat2(odds, preds, 0.05, validation_set))

At threshold 0.25 for predictions: 
 Profit: 83.20433333333328 
 Number of bets: 1860 
 Return %: 4.473351254480284 

At threshold 0.3 for predictions: 
 Profit: 124.12099999999995 
 Number of bets: 1657 
 Return %: 7.490706095353045 

At threshold 0.35 for predictions: 
 Profit: 146.62933333333334 
 Number of bets: 1467 
 Return %: 9.995182912974323 

At threshold 0.4 for predictions: 
 Profit: 139.781 
 Number of bets: 1279 
 Return %: 10.928928850664581 

At threshold 0.45 for predictions: 
 Profit: 143.16933333333338 
 Number of bets: 1109 
 Return %: 12.909768560264506 

At threshold 0.5 for predictions: 
 Profit: 134.03266666666678 
 Number of bets: 974 
 Return %: 13.761054072553058 

At threshold 0.55 for predictions: 
 Profit: 124.90433333333337 
 Number of bets: 840 
 Return %: 14.869563492063495 

At threshold 0.6 for predictions: 
 Profit: 98.78766666666671 
 Number of bets: 682 
 Return %: 14.484995112414472 

At threshold 0.65 for predictions: 
 Profit: 68.6933333333333 


In [62]:
X_train = train[features]
X_test = test[features]
y_train = train['home_win']
y_test = test['home_win']

model = LGBMClassifier()

kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(model, X_train, y_train, cv=kfold, verbose=True)

print("Results: ", results)
print("Mean: ", results.mean())
print("Std: ", results.std())

cv_results = cross_validate(model, X_train, y_train, cv=kfold, return_estimator=True)
trained_models = cv_results['estimator']
trained_model = model.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.6s finished


Results:  [0.71822199 0.71016376 0.71380296 0.70280811 0.70826833]
Mean:  0.7106530323079312
Std:  0.005189993417007074


In [63]:
X_train = train[features]
X_test = test[features]
y_train = train['home_win']
y_test = test['home_win']

def objective(trial):

    params = {
        'boosting_type': 'gbdt',
        'max_depth': trial.suggest_int('max_depth', 1, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=100),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0, step=0.1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0, step=0.1),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10, log=True)
    }

    model = LGBMClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)
    auc = np.mean(auc)
    obj_vector = np.array([accuracy, f1, precision, auc])
    print(obj_vector)
    return auc

study = optuna.create_study(directions=['maximize'])  
study.optimize(objective, n_trials=100)


best_params = study.best_params
best_model = LGBMClassifier(**best_params)
best_model.fit(X_train, y_train, early_stopping_rounds=1000,
                  eval_set=[(X_test, y_test)])

y_pred_test = best_model.predict(X_test)
y_prob_test = best_model.predict_proba(X_test)[:, 1] 

test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test)

test_roc_auc_score = roc_auc_score(y_test, y_prob_test)
test_precision_score = precision_score(y_test, y_pred_test)

print("Best Hyperparameters:", best_params)
print("Test Accuracy:", test_accuracy)
print("Test F1 Score:", test_f1)
print("Test auc:", test_roc_auc_score)
print("Test Precision Score:", test_precision_score)

model_filename = "lgbm_home_win_with_odds.pkl"
with open(model_filename, "wb") as file:
    pickle.dump(best_model, file)

[I 2023-06-30 16:35:41,832] A new study created in memory with name: no-name-b801ac68-d7b6-4f9d-8fd9-f0afa4402172
[I 2023-06-30 16:35:42,958] Trial 0 finished with value: 0.6799594219251801 and parameters: {'max_depth': 6, 'learning_rate': 0.0033957085450997343, 'n_estimators': 900, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.5, 'reg_lambda': 7.483684930461474e-05, 'reg_alpha': 0.0001549762589105797}. Best is trial 0 with value: 0.6799594219251801.


[0.68842365 0.62812347 0.69901854 0.67995942]


[I 2023-06-30 16:35:43,339] Trial 1 finished with value: 0.6889891285478691 and parameters: {'max_depth': 4, 'learning_rate': 0.08191744158464657, 'n_estimators': 600, 'subsample': 0.5, 'colsample_bytree': 0.5, 'reg_lambda': 0.00013597601715941752, 'reg_alpha': 9.930740700875111e-05}. Best is trial 1 with value: 0.6889891285478691.


[0.69499179 0.64903165 0.6918429  0.68898913]


[I 2023-06-30 16:35:44,016] Trial 2 finished with value: 0.6851727280618002 and parameters: {'max_depth': 4, 'learning_rate': 0.0047852502965763585, 'n_estimators': 800, 'subsample': 0.6000000000000001, 'colsample_bytree': 1.0, 'reg_lambda': 0.0017487553352383877, 'reg_alpha': 0.0023592832532525376}. Best is trial 1 with value: 0.6889891285478691.


...

[I 2023-06-30 16:36:50,333] Trial 99 finished with value: 0.6965486936897839 and parameters: {'max_depth': 5, 'learning_rate': 0.03127513623855104, 'n_estimators': 700, 'subsample': 0.6000000000000001, 'colsample_bytree': 0.7, 'reg_lambda': 4.1588797644185584, 'reg_alpha': 0.013310898396894963}. Best is trial 71 with value: 0.7097623361687353.


[0.70279146 0.65654649 0.70325203 0.69654869]
[1]	valid_0's binary_logloss: 0.685002
[2]	valid_0's binary_logloss: 0.680579
[3]	valid_0's binary_logloss: 0.676365
[4]	valid_0's binary_logloss: 0.671677
[5]	valid_0's binary_logloss: 0.667223
[6]	valid_0's binary_logloss: 0.663739
[7]	valid_0's binary_logloss: 0.660104
[8]	valid_0's binary_logloss: 0.656386
[9]	valid_0's binary_logloss: 0.653585
[10]	valid_0's binary_logloss: 0.650407
[11]	valid_0's binary_logloss: 0.647244
[12]	valid_0's binary_logloss: 0.644834
[13]	valid_0's binary_logloss: 0.642224
[14]	valid_0's binary_logloss: 0.639548
[15]	valid_0's binary_logloss: 0.63699
[16]	valid_0's binary_logloss: 0.63506
[17]	valid_0's binary_logloss: 0.632696
[18]	valid_0's binary_logloss: 0.63055
[19]	valid_0's binary_logloss: 0.628944
[20]	valid_0's binary_logloss: 0.626891
[21]	valid_0's binary_logloss: 0.624873
[22]	valid_0's binary_logloss: 0.623557
[23]	valid_0's binary_logloss: 0.621813
[24]	valid_0's binary_logloss: 0.620124
[25]	v

C:\Users\kiera\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[122]	valid_0's binary_logloss: 0.582086
[123]	valid_0's binary_logloss: 0.582069
[124]	valid_0's binary_logloss: 0.582048
[125]	valid_0's binary_logloss: 0.582089
[126]	valid_0's binary_logloss: 0.582056
[127]	valid_0's binary_logloss: 0.582099
[128]	valid_0's binary_logloss: 0.582095
[129]	valid_0's binary_logloss: 0.58209
[130]	valid_0's binary_logloss: 0.582048
[131]	valid_0's binary_logloss: 0.581884
[132]	valid_0's binary_logloss: 0.581853
[133]	valid_0's binary_logloss: 0.581813
[134]	valid_0's binary_logloss: 0.581809
[135]	valid_0's binary_logloss: 0.58162
[136]	valid_0's binary_logloss: 0.581579
[137]	valid_0's binary_logloss: 0.581597
[138]	valid_0's binary_logloss: 0.581305
[139]	valid_0's binary_logloss: 0.581463
[140]	valid_0's binary_logloss: 0.581438
[141]	valid_0's binary_logloss: 0.581293
[142]	valid_0's binary_logloss: 0.581236
[143]	valid_0's binary_logloss: 0.58128
[144]	valid_0's binary_logloss: 0.581267
[145]	valid_0's binary_logloss: 0.580992
[146]	valid_0's bin

[504]	valid_0's binary_logloss: 0.569398
[505]	valid_0's binary_logloss: 0.569414
[506]	valid_0's binary_logloss: 0.569378
[507]	valid_0's binary_logloss: 0.569234
[508]	valid_0's binary_logloss: 0.569145
[509]	valid_0's binary_logloss: 0.569133
[510]	valid_0's binary_logloss: 0.5691
[511]	valid_0's binary_logloss: 0.569032
[512]	valid_0's binary_logloss: 0.569002
[513]	valid_0's binary_logloss: 0.568963
[514]	valid_0's binary_logloss: 0.56891
[515]	valid_0's binary_logloss: 0.5689
[516]	valid_0's binary_logloss: 0.568888
[517]	valid_0's binary_logloss: 0.568911
[518]	valid_0's binary_logloss: 0.568863
[519]	valid_0's binary_logloss: 0.568844
[520]	valid_0's binary_logloss: 0.568792
[521]	valid_0's binary_logloss: 0.568807
[522]	valid_0's binary_logloss: 0.568813
[523]	valid_0's binary_logloss: 0.568832
[524]	valid_0's binary_logloss: 0.568793
[525]	valid_0's binary_logloss: 0.568768
[526]	valid_0's binary_logloss: 0.568755
[527]	valid_0's binary_logloss: 0.568735
[528]	valid_0's binar

In [58]:
thresholds = [0.25, 0.3, 0.35, 0.40, 0.45, 0.5, 0.55, 0.60, 0.65, .70, .75, 0.8, 0.85, 0.9]

for t in thresholds:
    preds = []
    for i in range(len(y_prob_test)):
        if y_prob_test[i] >= t:
            preds.append(1.)
        else:
            preds.append(0.)
    
    precision = precision_score(y_test, preds)
    recall = recall_score(y_test, preds)
    f1 = f1_score(y_test, preds)
    cm = confusion_matrix(y_test, preds)
    
    print(f"Threshold: {t}")
    print("Confusion Matrix:")
    print(cm)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print("-----------------------")

Threshold: 0.25
Confusion Matrix:
[[ 488  824]
 [ 101 1023]]
Precision: 0.5538711423930699
Recall: 0.9101423487544484
F1 Score: 0.6886570178391115
-----------------------
Threshold: 0.3
Confusion Matrix:
[[621 691]
 [157 967]]
Precision: 0.583232810615199
Recall: 0.8603202846975089
F1 Score: 0.6951833213515457
-----------------------
Threshold: 0.35
Confusion Matrix:
[[751 561]
 [227 897]]
Precision: 0.6152263374485597
Recall: 0.7980427046263345
F1 Score: 0.6948102246320682
-----------------------
Threshold: 0.4
Confusion Matrix:
[[859 453]
 [297 827]]
Precision: 0.64609375
Recall: 0.7357651245551602
F1 Score: 0.6880199667221298
-----------------------
Threshold: 0.45
Confusion Matrix:
[[949 363]
 [368 756]]
Precision: 0.675603217158177
Recall: 0.6725978647686833
F1 Score: 0.6740971912617031
-----------------------
Threshold: 0.5
Confusion Matrix:
[[1029  283]
 [ 435  689]]
Precision: 0.7088477366255144
Recall: 0.6129893238434164
F1 Score: 0.6574427480916032
-----------------------
Thr

In [59]:
validation_set = y_test.reset_index(drop = True)

for t in [0.25, 0.3, 0.35, 0.40, 0.45, 0.5, 0.55, 0.60, 0.65, .70, .75, 0.8, 0.85, 0.9]:
    results = get_profits(odds, y_prob_test, t, validation_set)
    print(f"At threshold {t} for predictions: \n Profit: {results[0]} \n Number of bets: {results[1]} \n Return %: {results[2]} \n")

print(get_value_strat1(odds, preds, 0.05, validation_set))
print(get_value_strat2(odds, preds, 0.05, validation_set))

At threshold 0.25 for predictions: 
 Profit: 139.7093333333334 
 Number of bets: 1847 
 Return %: 7.564121999639058 

At threshold 0.3 for predictions: 
 Profit: 159.76433333333335 
 Number of bets: 1658 
 Return %: 9.635967028548453 

At threshold 0.35 for predictions: 
 Profit: 161.61099999999996 
 Number of bets: 1458 
 Return %: 11.084430727023317 

At threshold 0.4 for predictions: 
 Profit: 175.05599999999998 
 Number of bets: 1280 
 Return %: 13.676249999999998 

At threshold 0.45 for predictions: 
 Profit: 154.776 
 Number of bets: 1119 
 Return %: 13.831635388739945 

At threshold 0.5 for predictions: 
 Profit: 146.9543333333334 
 Number of bets: 972 
 Return %: 15.118758573388211 

At threshold 0.55 for predictions: 
 Profit: 146.01433333333347 
 Number of bets: 836 
 Return %: 17.46582934609252 

At threshold 0.6 for predictions: 
 Profit: 128.82600000000002 
 Number of bets: 691 
 Return %: 18.643415340086833 

At threshold 0.65 for predictions: 
 Profit: 99.88833333333338 